In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Optuna는 딥러닝에서는 너무 느려서 안쓰고, 머신러닝에서 주로 쓴다, 최대 장점은 잘 몰라도 하이퍼 파라미터를 잘 찾아준다.
!pip install optuna

In [3]:
!pip install catboost

In [4]:
!pip install pycaret

In [5]:
#데이터 분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from lightgbm.sklearn import LGBMRegressor
from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor

# 상관관계 분석, VIF : 다중공산성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

# AutoML - pycaret
from pycaret.regression import *

In [6]:
# 코랩에서 한글 쓰기!
!sudo apt-get install -y fonts-nanum # 나눔 글꼴
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (6,971 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120899 files and dire

In [51]:
import matplotlib as mpl
# Colab 의 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic')
# 유니코드에서  음수 부호설정
mpl.rc('axes', unicode_minus=False) # False로 해야 -부호가 안깨짐

In [8]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/deacu_car_accident/train_all.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/deacu_car_accident/test.csv")

In [9]:
train

,ID,요일,기상상태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,...,주야구분,피해운전자 연령대,피해운전자 구분,가해운전자 연령대,가해운전자 구분,"시, 도",도로형태1,도로형태2,계절,사고 규모
0,COUNTRYWIDE_ACCIDENT_000000,화요일,맑음,건조,차대사람,횡단중,보행자보호의무위반,승용,남,26세,...,야간,40대,일반,20대,일반,서울특별시,교차로,교차로횡단보도내,겨울,소형
1,COUNTRYWIDE_ACCIDENT_000001,화요일,맑음,건조,차대차,기타,직진우회전진행방해,승용,남,54세,...,야간,20대,일반,50대,일반,경기도,교차로,교차로안,겨울,소형
2,COUNTRYWIDE_ACCIDENT_000002,화요일,맑음,건조,차대차,측면충돌,신호위반,승용,여,57세,...,야간,10대,일반,50대,일반,경기도,교차로,교차로안,겨울,중형
3,COUNTRYWIDE_ACCIDENT_000003,화요일,맑음,건조,차대사람,횡단중,보행자보호의무위반,승용,여,65세,...,야간,10대,일반,60대,고령,경기도,기타,기타,겨울,중형
4,COUNTRYWIDE_ACCIDENT_000004,화요일,맑음,건조,차대차,측면충돌,신호위반,승용,여,74세,...,야간,50대,일반,70대,고령,인천광역시,교차로,교차로안,겨울,중형
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616550,ACCIDENT_39604,금요일,맑음,건조,차대차,측면충돌,신호위반,승용,여,52세,...,야간,20대,일반,50대,일반,대구광역시,교차로,교차로안,겨울,소형
616551,ACCIDENT_39605,금요일,맑음,건조,차대차,측면충돌,안전거리미확보,승용,여,60세,...,야간,50대,일반,60대,고령,대구광역시,단일로,기타,겨울,소형
616552,ACCIDENT_39606,금요일,맑음,건조,차대차,측면충돌,교차로운행방법위반,승용,남,60세,...,야간,70대,고령,60대,고령,대구광역시,교차로,교차로안,겨울,중형
616553,ACCIDENT_39607,금요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,40세,...,야간,50대,일반,40대,일반,대구광역시,기타,기타,겨울,소형


In [10]:
test

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,ACCIDENT_39609,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람
1,ACCIDENT_39610,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람
2,ACCIDENT_39611,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차
3,ACCIDENT_39612,2022-01-01 04,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차
4,ACCIDENT_39613,2022-01-01 06,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차
...,...,...,...,...,...,...,...,...
10958,ACCIDENT_50567,2022-12-31 18,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차
10959,ACCIDENT_50568,2022-12-31 18,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차
10960,ACCIDENT_50569,2022-12-31 20,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차
10961,ACCIDENT_50570,2022-12-31 20,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차


## Data Wrangiling

In [11]:
def wrangiling(data_set):
    print("==================")
    print("       Shape      ")
    print("==================")

    display(data_set.shape)

    print("==================")
    print("        Info      ")
    print("==================")

    display(data_set.info())

    print("==================")
    print("      Columns     ")
    print("==================")

    display(data_set.columns)

    print("==================")
    print("        NaN       ")
    print("==================")

    display(data_set.isna().sum())

    print("==================")
    print("    Duplicated    ")
    print("==================")

    display(data_set[data_set.duplicated()])

    print("==================")
    print("    Description   ")
    print("==================")

    display(data_set.describe())

    print("==================")
    print("      Unique      ")
    print("==================")

    display(data_set.nunique())

In [12]:
wrangiling(train)

       Shape      


(616555, 33)

        Info      
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616555 entries, 0 to 616554
Data columns (total 33 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ID           616555 non-null  object
 1   요일           616555 non-null  object
 2   기상상태         616555 non-null  object
 3   노면상태         616555 non-null  object
 4   사고유형         616555 non-null  object
 5   사고유형 - 세부분류  616555 non-null  object
 6   법규위반         616555 non-null  object
 7   가해운전자 차종     616555 non-null  object
 8   가해운전자 성별     616555 non-null  object
 9   가해운전자 연령     616555 non-null  object
 10  가해운전자 상해정도   616555 non-null  object
 11  피해운전자 차종     615564 non-null  object
 12  피해운전자 성별     615564 non-null  object
 13  피해운전자 연령     615564 non-null  object
 14  피해운전자 상해정도   615564 non-null  object
 15  사망자수         616555 non-null  int64 
 16  중상자수         616555 non-null  int64 
 17  경상자수         616555 non-null  int64 
 18  부상자수         616555 non-n

None

      Columns     


...

,ID,요일,기상상태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,...,주야구분,피해운전자 연령대,피해운전자 구분,가해운전자 연령대,가해운전자 구분,"시, 도",도로형태1,도로형태2,계절,사고 규모


    Description   


,사망자수,중상자수,경상자수,부상자수,ECLO,월,일,시간
count,616555.000000,616555.000000,616555.000000,616555.000000,616555.000000,616555.000000,616555.000000,616555.000000
mean,0.011838,0.289119,1.096877,0.089857,4.944469,6.680921,15.833337,13.841388
std,0.112322,0.543976,1.046529,0.367386,3.570352,3.402979,8.762532,5.622692
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,3.000000,4.000000,8.000000,10.000000
50%,0.000000,0.000000,1.000000,0.000000,3.000000,7.000000,16.000000,14.000000
75%,0.000000,1.000000,1.000000,0.000000,6.000000,10.000000,23.000000,18.000000
max,6.000000,31.000000,41.000000,47.000000,245.000000,12.000000,31.000000,23.000000


      Unique      


ID             616555
요일                  7
기상상태                6
노면상태                7
사고유형                4
사고유형 - 세부분류        15
법규위반               11
가해운전자 차종           12
가해운전자 성별            3
가해운전자 연령           98
가해운전자 상해정도          6
피해운전자 차종           13
피해운전자 성별            3
피해운전자 연령          100
피해운전자 상해정도          6
사망자수                7
중상자수               19
경상자수               35
부상자수               21
ECLO              106
월                  12
일                  31
시간                 24
주야구분                2
피해운전자 연령대           9
피해운전자 구분            3
가해운전자 연령대           9
가해운전자 구분            3
시, 도               18
도로형태1               5
도로형태2              11
계절                  4
사고 규모               3
dtype: int64

In [13]:
def unique_cols(data_set):
    for col in data_set.columns[:]:
        print(f" == {col} ==")
        display(data_set[col].unique())

In [14]:
unique_cols(data_set=train)

 == ID ==


array(['COUNTRYWIDE_ACCIDENT_000000', 'COUNTRYWIDE_ACCIDENT_000001',
       'COUNTRYWIDE_ACCIDENT_000002', ..., 'ACCIDENT_39606',
       'ACCIDENT_39607', 'ACCIDENT_39608'], dtype=object)

 == 요일 ==


array(['화요일', '수요일', '목요일', '금요일', '토요일', '일요일', '월요일'], dtype=object)

 == 기상상태 ==


...

array([' 교차로횡단보도내', ' 교차로안', ' 기타', ' 교차로부근', ' 터널', ' 주차장', ' 고가도로위',
       ' 교량', ' 지하차도(도로)내', ' 미분류', ' 철길건널목'], dtype=object)

 == 계절 ==


array(['겨울', '봄', '여름', '가을'], dtype=object)

 == 사고 규모 ==


array(['소형', '중형', '대형'], dtype=object)

In [15]:
wrangiling(test)

       Shape      


(10963, 8)

        Info      
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10963 non-null  object
 1   사고일시    10963 non-null  object
 2   요일      10963 non-null  object
 3   기상상태    10963 non-null  object
 4   시군구     10963 non-null  object
 5   도로형태    10963 non-null  object
 6   노면상태    10963 non-null  object
 7   사고유형    10963 non-null  object
dtypes: object(8)
memory usage: 685.3+ KB


None

      Columns     


...

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형


    Description   


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
count,10963,10963,10963,10963,10963,10963,10963,10963
unique,10963,5548,7,5,192,11,6,3
top,ACCIDENT_39609,2022-11-10 18,금요일,맑음,대구광역시 남구 대명동,단일로 - 기타,건조,차대차
freq,1,10,1743,10321,422,5039,10394,8559


      Unique      


ID      10963
사고일시     5548
요일          7
기상상태        5
시군구       192
도로형태       11
노면상태        6
사고유형        3
dtype: int64

In [16]:
unique_cols(data_set=test)

 == ID ==


array(['ACCIDENT_39609', 'ACCIDENT_39610', 'ACCIDENT_39611', ...,
       'ACCIDENT_50569', 'ACCIDENT_50570', 'ACCIDENT_50571'], dtype=object)

 == 사고일시 ==


array(['2022-01-01 01', '2022-01-01 04', '2022-01-01 06', ...,
       '2022-12-31 18', '2022-12-31 20', '2022-12-31 21'], dtype=object)

 == 요일 ==


...

array(['교차로 - 교차로안', '단일로 - 기타', '교차로 - 교차로횡단보도내', '교차로 - 교차로부근',
       '단일로 - 지하차도(도로)내', '기타 - 기타', '단일로 - 교량', '단일로 - 고가도로위',
       '주차장 - 주차장', '단일로 - 터널', '미분류 - 미분류'], dtype=object)

 == 노면상태 ==


array(['건조', '젖음/습기', '서리/결빙', '기타', '침수', '적설'], dtype=object)

 == 사고유형 ==


array(['차대사람', '차대차', '차량단독'], dtype=object)

## Data Preprocessing for Modeling

In [17]:
test = test.drop("ID", axis = 1)

In [18]:
# train data의 columns 구성과 같게 만들어주는 작업

test["date_time"] = pd.to_datetime(test['사고일시'])
test['월'] = test.date_time.dt.month
test['일'] = test.date_time.dt.day
test['시간'] = test.date_time.dt.hour
test = test.drop(["사고일시"], axis = 1)
test = test.drop(["date_time"], axis = 1)

In [19]:
# 주, 야 구분 (6시~17시 주간, 18시~5시 야간)
test["주야구분"]=np.where(test["시간"].isin(range(6, 18)), "주간", "야간")
test

,요일,기상상태,시군구,도로형태,노면상태,사고유형,월,일,시간,주야구분
0,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람,1,1,1,야간
1,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람,1,1,1,야간
2,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차,1,1,4,야간
3,토요일,맑음,대구광역시 수성구 신매동,단일로 - 기타,건조,차대차,1,1,4,야간
4,토요일,맑음,대구광역시 달서구 감삼동,교차로 - 교차로안,건조,차대차,1,1,6,주간
...,...,...,...,...,...,...,...,...,...,...
10958,토요일,맑음,대구광역시 남구 대명동,단일로 - 터널,건조,차대차,12,31,18,야간
10959,토요일,맑음,대구광역시 수성구 시지동,단일로 - 기타,건조,차대차,12,31,18,야간
10960,토요일,맑음,대구광역시 수성구 연호동,단일로 - 기타,건조,차대차,12,31,20,야간
10961,토요일,맑음,대구광역시 수성구 범물동,교차로 - 교차로부근,건조,차대차,12,31,20,야간


In [20]:
# def split_age(age):
#     if age != np.nan:
#         if age < 10:
#             return "10대 미만"
#         elif age < 20:
#             return "10대"

#         elif age < 30:
#             return "20대"

#         elif age < 40:
#             return "30대"

#         elif age < 50:
#             return "40대"

#         elif age < 60:
#             return "50대"

#         elif age < 70:
#             return "60대"

#         elif age < 80:
#             return "70대"

#         else:
#             return "80대 이상"
#     else:
#         return "미분류"

In [21]:
# def age_category(age):
#         if age != np.nan:
#             if age < 10 :
#                 return "영유아"
#             elif age < 60 :
#                 return "일반"
#             else :
#                 return "고령"
#         else:
#              return "미분류"

In [22]:
# def age_add_variable(data:pd.DataFrame, age_col:str):
#     pattern = r'(\d+)'
#     data["temp"] = data[age_col].str.extract(pattern).astype(float)
#     data[age_col[:6]+"연령대"] = data["temp"].apply(split_age)
#     data[age_col[:6]+"구분"] = data["temp"].apply(age_category)
#     data.drop(["temp"], axis = 1, inplace = True)

In [23]:
# 연령대와 연령대에 따른 구분 # 연령이 안나와있어서 할 수 없음
# age_add_variable(data=test, age_col="피해운전자 연령")
# age_add_variable(data=test, age_col="가해운전자 연령")

In [24]:
# location extraction
def process_location_data(df, address_column, drop_columns):
    location_pattern = r'(\S+) (\S+)'
    df[['시,도', 'temp']] = df[address_column].str.extract(location_pattern)
    df = df.drop(['temp'], axis=1)
    df = df.drop(drop_columns, axis=1)
    return df

In [25]:
# 시, 도 구분 해봤자 대구광역시라 필요없음
# process_location_data(test, "시군구", "시군구")

In [26]:
test = test.drop("시군구", axis=1)

In [27]:
# 도로형태를 나눔
test[["도로형태1", "도로형태2"]] = test["도로형태"].str.split("-", expand=True)
test = test.drop(["도로형태"], axis=1)

In [28]:
def make_season (month):
    if 3 <= month and month <= 5:
        return "봄"
    elif 6 <= month and month <= 8:
        return "여름"
    elif 9 <= month and month <= 11:
        return "가을"
    else:
        return "겨울"

In [29]:
# 파생변수 "계절" 추가
test["계절"] = test["월"].apply(make_season)

In [30]:
# def accident_size(ECLO):
#     if ECLO < 5 :
#         return "소형"
#     elif 5 <= ECLO and ECLO <= 10 :
#         return "중형"
#     else :
#         return "대형"

In [31]:
# 사고 규모는 ECLO를 기반으로 작성하므로 test에서는 만들 수 없음
# test["사고 규모"] = test["ECLO"].apply(accident_size)

In [32]:
train

,ID,요일,기상상태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,...,주야구분,피해운전자 연령대,피해운전자 구분,가해운전자 연령대,가해운전자 구분,"시, 도",도로형태1,도로형태2,계절,사고 규모
0,COUNTRYWIDE_ACCIDENT_000000,화요일,맑음,건조,차대사람,횡단중,보행자보호의무위반,승용,남,26세,...,야간,40대,일반,20대,일반,서울특별시,교차로,교차로횡단보도내,겨울,소형
1,COUNTRYWIDE_ACCIDENT_000001,화요일,맑음,건조,차대차,기타,직진우회전진행방해,승용,남,54세,...,야간,20대,일반,50대,일반,경기도,교차로,교차로안,겨울,소형
2,COUNTRYWIDE_ACCIDENT_000002,화요일,맑음,건조,차대차,측면충돌,신호위반,승용,여,57세,...,야간,10대,일반,50대,일반,경기도,교차로,교차로안,겨울,중형
3,COUNTRYWIDE_ACCIDENT_000003,화요일,맑음,건조,차대사람,횡단중,보행자보호의무위반,승용,여,65세,...,야간,10대,일반,60대,고령,경기도,기타,기타,겨울,중형
4,COUNTRYWIDE_ACCIDENT_000004,화요일,맑음,건조,차대차,측면충돌,신호위반,승용,여,74세,...,야간,50대,일반,70대,고령,인천광역시,교차로,교차로안,겨울,중형
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616550,ACCIDENT_39604,금요일,맑음,건조,차대차,측면충돌,신호위반,승용,여,52세,...,야간,20대,일반,50대,일반,대구광역시,교차로,교차로안,겨울,소형
616551,ACCIDENT_39605,금요일,맑음,건조,차대차,측면충돌,안전거리미확보,승용,여,60세,...,야간,50대,일반,60대,고령,대구광역시,단일로,기타,겨울,소형
616552,ACCIDENT_39606,금요일,맑음,건조,차대차,측면충돌,교차로운행방법위반,승용,남,60세,...,야간,70대,고령,60대,고령,대구광역시,교차로,교차로안,겨울,중형
616553,ACCIDENT_39607,금요일,맑음,건조,차대차,추돌,안전운전불이행,승용,남,40세,...,야간,50대,일반,40대,일반,대구광역시,기타,기타,겨울,소형


In [33]:
test

,요일,기상상태,노면상태,사고유형,월,일,시간,주야구분,도로형태1,도로형태2,계절
0,토요일,맑음,건조,차대사람,1,1,1,야간,교차로,교차로안,겨울
1,토요일,맑음,건조,차대사람,1,1,1,야간,단일로,기타,겨울
2,토요일,맑음,건조,차대차,1,1,4,야간,교차로,교차로안,겨울
3,토요일,맑음,건조,차대차,1,1,4,야간,단일로,기타,겨울
4,토요일,맑음,건조,차대차,1,1,6,주간,교차로,교차로안,겨울
...,...,...,...,...,...,...,...,...,...,...,...
10958,토요일,맑음,건조,차대차,12,31,18,야간,단일로,터널,겨울
10959,토요일,맑음,건조,차대차,12,31,18,야간,단일로,기타,겨울
10960,토요일,맑음,건조,차대차,12,31,20,야간,단일로,기타,겨울
10961,토요일,맑음,건조,차대차,12,31,20,야간,교차로,교차로부근,겨울


In [34]:
# test에 있는 columns만 사용

test.columns

Index(['요일', '기상상태', '노면상태', '사고유형', '월', '일', '시간', '주야구분', '도로형태1', '도로형태2',
       '계절'],
      dtype='object')

In [35]:
use_columns = ['요일', '기상상태', '노면상태', '사고유형', '월', '일', '시간', '주야구분', '도로형태1', '도로형태2', '계절', 'ECLO']

In [36]:
train = train[use_columns]
train

,요일,기상상태,노면상태,사고유형,월,일,시간,주야구분,도로형태1,도로형태2,계절,ECLO
0,화요일,맑음,건조,차대사람,1,1,0,야간,교차로,교차로횡단보도내,겨울,3
1,화요일,맑음,건조,차대차,1,1,0,야간,교차로,교차로안,겨울,3
2,화요일,맑음,건조,차대차,1,1,0,야간,교차로,교차로안,겨울,5
3,화요일,맑음,건조,차대사람,1,1,0,야간,기타,기타,겨울,10
4,화요일,맑음,건조,차대차,1,1,0,야간,교차로,교차로안,겨울,7
...,...,...,...,...,...,...,...,...,...,...,...,...
616550,금요일,맑음,건조,차대차,12,31,19,야간,교차로,교차로안,겨울,3
616551,금요일,맑음,건조,차대차,12,31,19,야간,단일로,기타,겨울,3
616552,금요일,맑음,건조,차대차,12,31,21,야간,교차로,교차로안,겨울,10
616553,금요일,맑음,건조,차대차,12,31,22,야간,기타,기타,겨울,3


In [37]:
# 데이터 확인 작업

In [38]:
train.isna().sum()

요일       0
기상상태     0
노면상태     0
사고유형     0
월        0
일        0
시간       0
주야구분     0
도로형태1    0
도로형태2    0
계절       0
ECLO     0
dtype: int64

In [39]:
test.isna().sum()

요일       0
기상상태     0
노면상태     0
사고유형     0
월        0
일        0
시간       0
주야구분     0
도로형태1    0
도로형태2    0
계절       0
dtype: int64

## Modeling

In [40]:
# # Define a function to calculate RMSLE
# from sklearn.metrics import make_scorer
# def rmsle(y_true, y_pred):
#     return np.sqrt(np.mean(np.square(np.log1p(y_pred) - np.log1p(y_true))))

# # # Create a scorer for RMSLE
# # rmsle_scorer = make_scorer(rmsle, greater_is_better=False)

In [41]:
train.columns

Index(['요일', '기상상태', '노면상태', '사고유형', '월', '일', '시간', '주야구분', '도로형태1', '도로형태2',
       '계절', 'ECLO'],
      dtype='object')

In [46]:
exp = setup(session_id=42, data=train, train_size=0.8, fold_shuffle=True, data_split_shuffle=True)

,Description,Value
0,Session id,42
1,Target,ECLO
2,Target type,Regression
3,Original data shape,"(616555, 12)"
4,Transformed data shape,"(616555, 49)"
5,Transformed train set shape,"(493244, 49)"
6,Transformed test set shape,"(123311, 49)"
7,Ordinal features,1
8,Numeric features,3
9,Categorical features,8


In [49]:
best_model = compare_models(n_select=25, sort='RMSLE', include = ['lasso', 'ridge', 'lightgbm', 'xgboost', 'catboost', 'dt', 'lr', 'rf', 'gbr'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,2.2386,12.3492,3.5132,0.0260,0.4549,0.5882,13.5860
gbr,Gradient Boosting Regressor,2.2368,12.3588,3.5146,0.0252,0.4550,0.5882,57.1810
catboost,CatBoost Regressor,2.2414,12.3960,3.5199,0.0222,0.4555,0.5884,56.4250
xgboost,Extreme Gradient Boosting,2.2436,12.4119,3.5221,0.0210,0.4560,0.5888,8.6540
ridge,Ridge Regression,2.2431,12.3949,3.5197,0.0224,0.4562,0.5899,3.2160
lr,Linear Regression,2.2431,12.3949,3.5197,0.0224,0.4562,0.5899,4.3170
lasso,Lasso Regression,2.2324,12.6777,3.5597,-0.0000,0.4611,0.5883,3.7000
rf,Random Forest Regressor,2.4880,14.9792,3.8696,-0.1819,0.5054,0.6413,289.5260
dt,Decision Tree Regressor,2.6450,17.8909,4.2292,-0.4122,0.5469,0.6708,7.5630


Processing:   0%|          | 0/49 [00:00<?, ?it/s]

from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('  ').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [53]:
lightgbm_model = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.2401,12.1473,3.4853,0.0260,0.4567,0.5947
1,2.2222,12.3139,3.5091,0.0247,0.4531,0.5855
2,2.2407,11.9943,3.4633,0.0276,0.4547,0.5884
3,2.2330,11.5828,3.4033,0.0271,0.4551,0.5915
4,2.2491,13.4662,3.6696,0.0232,0.4553,0.5859
5,2.2472,11.9010,3.4498,0.0269,0.4559,0.5884
6,2.2417,12.9264,3.5953,0.0250,0.4550,0.5873
7,2.2396,11.6639,3.4152,0.0288,0.4540,0.5864
8,2.2385,12.6369,3.5548,0.0256,0.4545,0.5857


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [54]:
catboost_model = create_model('catboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.2421,12.1929,3.4918,0.0223,0.4574,0.5947
1,2.2277,12.5103,3.5370,0.0091,0.4541,0.5867
2,2.2433,12.0201,3.4670,0.0255,0.4552,0.5884
3,2.2357,11.6162,3.4082,0.0243,0.4557,0.5916
4,2.2504,13.4832,3.6719,0.0219,0.4557,0.5856
5,2.2510,11.9478,3.4566,0.0231,0.4567,0.5890
6,2.2438,12.9494,3.5985,0.0232,0.4555,0.5872
7,2.2419,11.6892,3.4189,0.0267,0.4546,0.5866
8,2.2410,12.6591,3.5580,0.0239,0.4551,0.5859


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [55]:
gbr_model = create_model('gbr')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.2391,12.1580,3.4868,0.0251,0.4570,0.5949
1,2.2203,12.3314,3.5116,0.0233,0.4533,0.5854
2,2.2394,11.9994,3.4640,0.0272,0.4548,0.5885
3,2.2312,11.5962,3.4053,0.0260,0.4554,0.5915
4,2.2476,13.4821,3.6718,0.0220,0.4555,0.5858
5,2.2454,11.9099,3.4511,0.0262,0.4561,0.5884
6,2.2394,12.9241,3.5950,0.0251,0.4550,0.5872
7,2.2378,11.6774,3.4172,0.0277,0.4540,0.5861
8,2.2363,12.6430,3.5557,0.0251,0.4546,0.5855


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [56]:
xgboost_model = create_model('xgboost')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.2445,12.2085,3.4941,0.0211,0.4578,0.5950
1,2.2297,12.4468,3.5280,0.0142,0.4546,0.5868
2,2.2449,12.0390,3.4697,0.0240,0.4555,0.5884
3,2.2388,11.6448,3.4125,0.0219,0.4564,0.5924
4,2.2523,13.5086,3.6754,0.0201,0.4561,0.5859
5,2.2520,11.9542,3.4575,0.0226,0.4570,0.5892
6,2.2474,12.9852,3.6035,0.0205,0.4563,0.5880
7,2.2450,11.7460,3.4272,0.0219,0.4552,0.5872
8,2.2433,12.6763,3.5604,0.0225,0.4555,0.5865


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [58]:
ridge_model = create_model('ridge')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.2461,12.1982,3.4926,0.0219,0.4583,0.5969
1,2.2259,12.3519,3.5145,0.0217,0.4542,0.5865
2,2.2442,12.0289,3.4683,0.0248,0.4557,0.5897
3,2.2385,11.6407,3.4118,0.0223,0.4568,0.5935
4,2.2532,13.5247,3.6776,0.0189,0.4567,0.5873
5,2.2519,11.9463,3.4563,0.0232,0.4572,0.5903
6,2.2462,12.9619,3.6003,0.0223,0.4562,0.5891
7,2.2434,11.7126,3.4224,0.0247,0.4551,0.5876
8,2.2425,12.6736,3.5600,0.0227,0.4558,0.5875


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [61]:
lightgbm_tuned_model = tune_model(lightgbm_model, n_iter = 5, optimize='RMSLE')
catboost_tuned_model = tune_model(catboost_model, n_iter = 5, optimize='RMSLE')
gbr_tuned_model = tune_model(gbr_model, n_iter = 5, optimize='RMSLE')
xgboost_tuned_model = tune_model(xgboost_model, n_iter = 5, optimize='RMSLE')
ridge_tuned_model = tune_model(ridge_model, n_iter = 5, optimize='RMSLE')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.2414,12.1710,3.4887,0.0241,0.4570,0.5947
1,2.2239,12.3250,3.5107,0.0238,0.4534,0.5858
2,2.2418,11.9982,3.4638,0.0273,0.4547,0.5882
3,2.2349,11.6026,3.4063,0.0255,0.4555,0.5917
4,2.2511,13.4777,3.6712,0.0223,0.4555,0.5862
5,2.2498,11.9126,3.4515,0.0260,0.4562,0.5888
6,2.2437,12.9520,3.5989,0.0230,0.4553,0.5877
7,2.2403,11.6701,3.4161,0.0283,0.4541,0.5863
8,2.2395,12.6405,3.5553,0.0253,0.4546,0.5857


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 5 candidates, totalling 50 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.2397,12.1541,3.4863,0.0254,0.4570,0.5951
1,2.2206,12.3108,3.5087,0.0249,0.4531,0.5853
2,2.2401,11.9913,3.4628,0.0278,0.4547,0.5887
3,2.2327,11.5954,3.4052,0.0261,0.4554,0.5919
4,2.2489,13.4808,3.6716,0.0221,0.4555,0.5862
5,2.2465,11.9019,3.4499,0.0268,0.4559,0.5885
6,2.2403,12.9183,3.5942,0.0256,0.4549,0.5873
7,2.2385,11.6673,3.4157,0.0285,0.4539,0.5863
8,2.2375,12.6341,3.5545,0.0258,0.4545,0.5858


...

Fitting 10 folds for each of 5 candidates, totalling 50 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.2460,12.1975,3.4925,0.0220,0.4583,0.5969
1,2.2259,12.3517,3.5145,0.0217,0.4542,0.5865
2,2.2442,12.0291,3.4683,0.0248,0.4557,0.5897
3,2.2384,11.6407,3.4118,0.0223,0.4568,0.5935
4,2.2531,13.5244,3.6775,0.0189,0.4566,0.5872
5,2.2519,11.9467,3.4564,0.0232,0.4572,0.5903
6,2.2461,12.9618,3.6002,0.0223,0.4562,0.5891
7,2.2433,11.7131,3.4224,0.0247,0.4551,0.5876
8,2.2425,12.6737,3.5600,0.0227,0.4558,0.5875


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 5 candidates, totalling 50 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [62]:
blender5_model = blend_models(estimator_list=[lightgbm_tuned_model,
                                              catboost_tuned_model,
                                              gbr_tuned_model,
                                              xgboost_tuned_model,
                                              ridge_tuned_model])

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.2041,12.1958,3.4923,0.0221,0.4497,0.5652
1,2.1881,12.3615,3.5159,0.0209,0.4461,0.5564
2,2.2061,12.0504,3.4714,0.0231,0.4479,0.5593
3,2.1981,11.6366,3.4112,0.0226,0.4483,0.5624
4,2.2139,13.5333,3.6788,0.0183,0.4485,0.5565
5,2.2119,11.9575,3.4580,0.0223,0.4491,0.5593
6,2.2054,12.9773,3.6024,0.0211,0.4480,0.5581
7,2.2049,11.7249,3.4242,0.0237,0.4471,0.5572
8,2.2034,12.6899,3.5623,0.0215,0.4476,0.5567


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [63]:
final_model = finalize_model(blender5_model)

In [70]:
prediction = predict_model(final_model, data = test)

In [71]:
prediction

,요일,기상상태,노면상태,사고유형,월,일,시간,주야구분,도로형태1,도로형태2,계절,prediction_label
0,토요일,맑음,건조,차대사람,1,1,1,야간,교차로,교차로안,겨울,4.464509
1,토요일,맑음,건조,차대사람,1,1,1,야간,단일로,기타,겨울,4.283347
2,토요일,맑음,건조,차대차,1,1,4,야간,교차로,교차로안,겨울,5.548768
3,토요일,맑음,건조,차대차,1,1,4,야간,단일로,기타,겨울,5.381038
4,토요일,맑음,건조,차대차,1,1,6,주간,교차로,교차로안,겨울,5.336191
...,...,...,...,...,...,...,...,...,...,...,...,...
10958,토요일,맑음,건조,차대차,12,31,18,야간,단일로,터널,겨울,6.661228
10959,토요일,맑음,건조,차대차,12,31,18,야간,단일로,기타,겨울,5.025676
10960,토요일,맑음,건조,차대차,12,31,20,야간,단일로,기타,겨울,5.037821
10961,토요일,맑음,건조,차대차,12,31,20,야간,교차로,교차로부근,겨울,4.999044


In [82]:
nice = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/deacu_car_accident/35.csv")
nice

,ID,ECLO
0,ACCIDENT_39609,3.762969
1,ACCIDENT_39610,3.446971
2,ACCIDENT_39611,4.614977
3,ACCIDENT_39612,4.096889
4,ACCIDENT_39613,4.441886
...,...,...
10958,ACCIDENT_50567,4.753187
10959,ACCIDENT_50568,4.383807
10960,ACCIDENT_50569,4.846968
10961,ACCIDENT_50570,4.727992


In [83]:
import math

pred = pd.DataFrame({
    "ECLO" : prediction["prediction_label"],
    "ECLO-Best" : nice["ECLO"],
    "ECLO-Round UP" : prediction["prediction_label"].apply(math.ceil),
    "ECLO-Round Down" : prediction["prediction_label"].apply(math.floor),
    "ECLO-Round" : prediction["prediction_label"].apply(round)
    })

In [84]:
pred

,ECLO,ECLO-Best,ECLO-Round UP,ECLO-Round Down,ECLO-Round
0,4.464509,3.762969,5,4,4
1,4.283347,3.446971,5,4,4
2,5.548768,4.614977,6,5,6
3,5.381038,4.096889,6,5,5
4,5.336191,4.441886,6,5,5
...,...,...,...,...,...
10958,6.661228,4.753187,7,6,7
10959,5.025676,4.383807,6,5,5
10960,5.037821,4.846968,6,5,5
10961,4.999044,4.727992,5,4,5


In [85]:
pred.to_excel("my_model_score.xlsx")